#  Dataset Preparation

## Creating "TVT" folder with train and test and validation folder and Happy and Sad folder inside them


In [15]:
import os

currentDir = os.getcwd()
TVTFolder = os.path.join(currentDir, "TVT")

TVT = ["Train", "Validation", "Test"]
HS = ["Happy", "Sad"]

for main in TVT:
    for sub in HS:
        os.makedirs(os.path.join(TVTFolder, main,sub), exist_ok= True)

## Split the dataset in to Train, Test and Validation folders

In [16]:
import os
import shutil
import random

currentDir = os.getcwd()

trainRatio = 0.7
valRatio = 0.15
testRatio = 0.15

HS = ["Happy", "Sad"]

for file in HS:
    folderDir = os.path.join(currentDir, "data", file)
    images = os.listdir(folderDir)

    random.shuffle(images)

    imagesLen = len(images)
    nTrain = int(imagesLen * trainRatio)
    nVal = int(imagesLen * valRatio)

    # Split
    trainFiles = images[:nTrain]
    valFiles = images[nTrain:nTrain + nVal]
    testFiles = images[nTrain + nVal:]

    # Copy to Train
    for fname in trainFiles:
        shutil.copy(os.path.join(folderDir, fname),
                    os.path.join("TVT", "Train", file, fname))

    # Copy to Val
    for fname in valFiles:
        shutil.copy(os.path.join(folderDir, fname),
                    os.path.join("TVT", "Validation", file, fname))

    # Copy to Test
    for fname in testFiles:
        shutil.copy(os.path.join(folderDir, fname),
                    os.path.join("TVT", "Test", file, fname))

print("Dataset split into Train/Val/Test successfully!")

Dataset split into Train/Val/Test successfully!


# Preprocessing

## For handeling ICC profile


In [17]:
import os
from PIL import Image

currentDir = os.getcwd()
HS = ["Happy", "Sad"]
TVT = ["Train", "Validation", "Test"]
valid_extensions = (".png", ".jpg", ".jpeg")

for TVTfolders in TVT:
    for category in HS:
        folderDir = os.path.join(currentDir, "TVT", TVTfolders, category)

        for picture in os.listdir(folderDir):
            if not picture.lower().endswith(valid_extensions):
                continue  # skip non-image files

            picturePath = os.path.join(folderDir, picture)

            try:
                img = Image.open(picturePath)
                
                if img.mode != 'RGB':
                    img = img.convert('RGB')

                img.save(picturePath)
            except Exception as e:
                print(f"Skipping {picturePath}: {e}")
            
        

/Users/parsaasil/miniforge3/envs/tf_m3/lib/python3.10/site-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


## Changing names that start with sad or happy 

In [18]:
import os

currentDir = os.getcwd()

HS = ["Happy", "Sad"]
TVT = ["Train", "Validation", "Test"]

fileNumber = 0

for TVTfolder in TVT:
    for folder in HS:
        FolderDir = os.path.join(currentDir, "TVT",TVTfolder, folder)
    
          
    
        for picture in os.listdir(FolderDir):
            fileName, fileType = os.path.splitext(picture)
    
            newName = f"{folder}_{fileNumber}{fileType}"
            os.rename(
                os.path.join(FolderDir, picture),
                os.path.join(FolderDir, newName)
            )
    
            
    
            fileNumber += 1

## Crop Faces

In [19]:
from mtcnn import MTCNN
import cv2
import os

detector = MTCNN()
currentDir = os.getcwd()
HS = ["Happy", "Sad"]
TVT = ["Train", "Validation", "Test"]

for split in TVT:
    for label in HS:
        folderDir = os.path.join(currentDir, "TVT", split, label)
        for img_name in os.listdir(folderDir):
            img_path = os.path.join(folderDir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            results = detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if results:
                x, y, w, h = results[0]['box']
                face_img = img[y:y+h, x:x+w]
                cv2.imwrite(img_path, face_img)  # Overwrite or save elsewhere

libpng warning: iCCP: known incorrect sRGB profile


## Resize, Normalize, data augmentation

### Resize, Normalize

In [20]:
import os
import cv2
import numpy as np

currentDir = os.getcwd()
HS = ["Happy", "Sad"]
TVT = ["Train", "Validation", "Test"]
validExtensions = (".png", ".jpg", ".jpeg")

TVTFolderDir = os.path.join(currentDir, "TVT")

images = []
labels = []
splits = []  # To keep track of whether it's Train/Validation/Test

for split in TVT:  # Train, Validation, Test
    splitDir = os.path.join(TVTFolderDir, split)

    for idx, dataFolder in enumerate(HS):  # Happy, Sad
        dataFolderDir = os.path.join(splitDir, dataFolder)

        if not os.path.exists(dataFolderDir):
            print(f"Skipping missing folder: {dataFolderDir}")
            continue

        for picture in os.listdir(dataFolderDir):
            _, fileType = os.path.splitext(picture)

            if fileType.lower() in validExtensions:
                pictureDir = os.path.join(dataFolderDir, picture)
                img = cv2.imread(pictureDir)

                if img is None:
                    print(f"Failed to load {pictureDir}")
                    continue

                # Resize
                img = cv2.resize(img, (224, 224))

                # Normalize
                img = img.astype(np.float32) / 255.0

                # Store
                images.append(img)
                labels.append(idx)   # 0 for Happy, 1 for Sad
                splits.append(split) # Train / Validation / Test

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)
splits = np.array(splits)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)
print("Splits shape:", splits.shape)

libpng warning: iCCP: known incorrect sRGB profile


Images shape: (1988, 224, 224, 3)
Labels shape: (1988,)
Splits shape: (1988,)


### Data augmentation

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Select only training data
train_images = images[splits == "Train"]
train_labels = labels[splits == "Train"]

# Optionally, validation data (no augmentation)
val_images = images[splits == "Validation"]
val_labels = labels[splits == "Validation"]

# Create the data augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Fit the generator (optional, only needed for featurewise normalization)
train_datagen.fit(train_images)

# Create iterator
batch_size = 16
train_generator = train_datagen.flow(train_images, train_labels, batch_size=batch_size)
val_generator = ImageDataGenerator().flow(val_images, val_labels, batch_size=batch_size)


# Example: get one batch
augmented_images, augmented_labels = next(train_generator)
print("Augmented batch shape:", augmented_images.shape)
print("Augmented labels shape:", augmented_labels.shape)

Augmented batch shape: (16, 224, 224, 3)
Augmented labels shape: (16,)


# Model

In [22]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Load VGG16 without top layers (we'll add our own classifier)
base_model = VGG16(weights='models/vgg16_notop.h5', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers so we don't train them yet
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: Happy, Sad
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',  # because labels are integers
              metrics=['accuracy'])

# Train using your data generator
batch_size = 16
history = model.fit(
    train_generator,
    #steps_per_epoch=len(train_images)//batch_size,
    validation_data=val_generator,
    #validation_steps=len(val_images)//batch_size,
    epochs=20
)

Epoch 1/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 271ms/step - accuracy: 0.5651 - loss: 1.1706 - val_accuracy: 0.7239 - val_loss: 0.5537
Epoch 2/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 330ms/step - accuracy: 0.6211 - loss: 1.0950 - val_accuracy: 0.7475 - val_loss: 0.5451
Epoch 3/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 331ms/step - accuracy: 0.6736 - loss: 0.9854 - val_accuracy: 0.7677 - val_loss: 0.4955
Epoch 4/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 26s 302ms/step - accuracy: 0.6592 - loss: 1.0281 - val_accuracy: 0.7879 - val_loss: 0.4931
Epoch 5/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 24s 281ms/step - accuracy: 0.6556 - loss: 0.9699 - val_accuracy: 0.7912 - val_loss: 0.4975
Epoch 6/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 25s 284ms/step - accuracy: 0.6887 - loss: 0.9334 - val_accuracy: 0.7710 - val_loss: 0.5575
Epoch 7/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 26s 301ms/step - accuracy: 0.7009 - loss: 0.8845 - val_accuracy: 0.7946 - val_loss: 0.5019
Epoch 8/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 27s 305ms/step - accuracy: 0.6952 - loss: 0.9861 - val_accu

In [ ]:
# Save the entire model
model.save("vgg16_emotion_model.h5")
print("Model saved successfully!")

Model saved successfully!


# Testing

## Testing all "Test" folder pictures

In [ ]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from tensorflow.keras.models import load_model

currentDir = os.getcwd()
test_dir = os.path.join(currentDir, "TVT", "Test")


# Load model
model = load_model("vgg16_emotion_model.h5")  # replace with your model path

# Test folder path
classes = ["Happy", "Sad"]

detector = MTCNN()

X_test = []
y_test = []

for idx, cls in enumerate(classes):
    folder = os.path.join(test_dir, cls)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        # Detect face
        results = detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if results:
            x, y, w, h = results[0]['box']
            x, y = max(0, x), max(0, y)  # avoid negative values
            face_img = img[y:y+h, x:x+w]
            face_img = cv2.resize(face_img, (224, 224))
            face_img = face_img.astype(np.float32) / 255.0
            X_test.append(face_img)
            y_test.append(idx)

X_test = np.array(X_test)
y_test = np.array(y_test)

# Predict
preds = model.predict(X_test)
pred_classes = np.argmax(preds, axis=1)

# Calculate accuracy
accuracy = np.sum(pred_classes == y_test) / len(y_test)
print("Test Accuracy:", accuracy)

2025-09-03 09:38:49.099973: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-09-03 09:38:49.099998: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-03 09:38:49.100003: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-03 09:38:49.100177: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-03 09:38:49.100185: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-09-03 09:39:05.167124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 304ms/step
Test Accuracy: 0.8253968253968254


## Testing specific picture

In [2]:
from mtcnn import MTCNN
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model

detector = MTCNN()
currentDir = os.getcwd()

model = load_model("vgg16_emotion_model2.h5")  # replace with your file path
img_name = "333.jpg"  # put your image name here
folderDir = os.path.join(currentDir, "TestImage")  # corrected variable name
img_path = os.path.join(folderDir, img_name)

img = cv2.imread(img_path)

if img is None:
    print("Image not found!")
else:
    results = detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if results:
        x, y, w, h = results[0]['box']
        face_img = img[y:y+h, x:x+w]

        face_img = cv2.resize(face_img, (224, 224))

        # Normalize
        face_img = face_img.astype(np.float32) / 255.0

        # Add batch dimension
        face_img = np.expand_dims(face_img, axis=0)  # shape becomes (1, 224, 224, 3)
        
        # Predict
        pred = model.predict(face_img)
        pred_class = np.argmax(pred, axis=1)[0]
        
        # Map to labels
        labels = ["Happy", "Sad"]
        print("Predicted:", labels[pred_class])

        
    else:
        print("No face detected in the image.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Predicted: Happy
